In [1]:
pip install torch==1.11.0+cu113 torchvision==0.12.0+cu113 torchaudio==0.11.0 torchtext==0.12.0 --extra-index-url https://download.pytorch.org/whl/cu113

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://download.pytorch.org/whl/cu113
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 1.6/1.6 GB 85.7 MB/s eta 0:00:01tcmalloc: large alloc 1636999168 bytes == 0x32c2000 @  0x7f7a4301d1e7 0x4d30a0 0x4d312c 0x5d6f4c 0x51edd1 0x51ef5b 0x4f750a 0x4997a2 0x55cd91 0x5d8941 0x4997a2 0x55cd91 0x5d8941 0x4997a2 0x55cd91 0x5d8941 0x4997a2 0x55cd91 0x5d8941 0x4997a2 0x55cd91 0x5d8941 0x4997a2 0x5d8868 0x4997a2 0x55cd91 0x5d8941 0x49abe4 0x55cd91 0x5d8941 0x4997a2
tcmalloc: large alloc 2046255104 bytes == 0x64bec000 @  0x7f7a4301e615 0x5d6f4c 0x51edd1 0x51ef5b 0x4f750a 0x4997a2 0x55cd91 0x5d8941 0x4997a2 0x55cd91 0x5d8941 0x4997a2 0x55cd91 0x5d8941 0x4997a2 0x55cd91 0x5d8941 0x4997a2 0x55cd91 0x5d8941 0x4997a2 0x5d8868 0x4997a2 0x55cd91 0x5d8941 0x49abe4 0x55cd91 0x5d8941 0x4997a2 0x55cd91 0x5d8941
tcmalloc: large alloc 1636999168 bytes == 0x32c2000 @  0x7f7a4301d1e7 0x4d30a0 0x5dede2 0x6758aa 0x4f7

In [2]:
import scipy
import torch
import matplotlib.pyplot as plt

In [3]:
# To install Falkon library
!pip install git+https://github.com/falkonml/falkon.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/falkonml/falkon.git to /tmp/pip-req-build-n3y5u0yd
  Running command git clone --filter=blob:none --quiet https://github.com/falkonml/falkon.git /tmp/pip-req-build-n3y5u0yd
  Resolved https://github.com/falkonml/falkon.git to commit e0f35851b00e181d6f2a5c52fc1aa160b1e7b0d6
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/getkeops/keops (to revision ad044a671fdc3c2790b0321f6b9f9b5aa3d220df) to /tmp/pip-install-r5necui7/keopscore_27e4d01669a844b68b6198cb3179f350
  Running command git clone --filter=blob:none --quiet https://github.com/getkeops/keops /tmp/pip-install-r5necui7/keopscore_27e4d01669a844b68b6198cb3179f350
  Running command git rev-parse -q --verify 'sha^ad044a671fdc3c2790b0321f6b9f9b5aa3d220df'
  Running command git fetch -q https://github.com/getkeops/keops ad044a671fdc3c2790b0321f6b9f9b5aa3d220df
  Running command git checkout 

In [4]:
import falkon

[KeOps] Warning : cuda was detected, but driver API could not be initialized. Switching to cpu only.


In [5]:
# Drive
from google.colab import drive

# we connect to the drive folder to work with the data stored
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import numpy as np
import pandas as pd

In [7]:
import os, time

In [8]:
import numpy as np

import os, time

import torch

from falkon import LogisticFalkon
from falkon.kernels import GaussianKernel
from falkon.options import FalkonOptions
from falkon.gsc_losses import WeightedCrossEntropyLoss

import matplotlib.pyplot as plt
import matplotlib.font_manager as font_manager

from sklearn import metrics
from scipy.spatial.distance import pdist
from scipy.stats import norm, chi2, rv_continuous, kstest


# UTILS

def candidate_sigma(data, perc=90):
    # this function estimates the width of the gaussian kernel.
    # use on a (small) sample of reference data (standardize first if necessary)
    pairw = pdist(data)
    return round(np.percentile(pairw,perc),1)

'''
def NP2_gen(size, seed):
    # custom function to generate samples of non-resonant new physics events
    if size>10000:
        raise Warning('Sample size is grater than 1000: Generator will not approximate the tail well')
    sample = np.array([])
    #normalization factor                                                                                                                                    
    np.random.seed(seed)
    Norm = 256.*0.25*0.25*np.exp(-2)
    while(len(sample)<size):
        x = np.random.uniform(0,1) #assuming not to generate more than 10 000 events                                                                         
        p = np.random.uniform(0, Norm)
        if p<= 256.*x*x*np.exp(-8.*x):
            sample = np.append(sample, x)
    return sample
'''

class non_res(rv_continuous):

    def _pdf(self, x):

        return 256 * (x**2) * np.exp(- 8 * x)

def nonres_sig(N_S, seed):
    # this function can be used to generate non-resonant signal events.
    
    my_sig = non_res(momtype = 0, a=0, b=1, seed=seed)
    
    sig_sample = my_sig.rvs(size = N_S)
    
    return sig_sample


def get_logflk_config(M,flk_sigma,lam,weight,iter=[100],seed=None,cpu=False):
    # it returns logfalkon parameters
    return {
            'kernel' : GaussianKernel(sigma=flk_sigma),
            'M' : M, #number of Nystrom centers,
            'penalty_list' : lam, # list of regularization parameters,
            'iter_list' : iter, #list of number of CG iterations,
            'options' : FalkonOptions(cg_tolerance=np.sqrt(1e-7), keops_active='no', use_cpu=cpu, debug = False),
            'seed' : seed, # (int or None), the model seed (used for Nystrom center selection) is manually set,
            'loss' : WeightedCrossEntropyLoss(kernel=GaussianKernel(sigma=flk_sigma), neg_weight=weight),
            }


def compute_t(preds,Y,weight):
    # it returns extended log likelihood ratio from predictions
    diff = weight*np.sum(1 - np.exp(preds[Y==0]))
    return 2 * (diff + np.sum(preds[Y==1]))

def trainer(X,Y,flk_config):
    # trainer for logfalkon model
    Xtorch=torch.from_numpy(X)
    Ytorch=torch.from_numpy(Y)
    model = LogisticFalkon(**flk_config)
    model.fit(Xtorch, Ytorch)
    return model.predict(Xtorch).numpy()

def standardize(X):
    # standardize data as in HIGGS and SUSY
    for j in range(X.shape[1]):
        column = X[:, j]

        mean = np.mean(column)
        std = np.std(column)
    
        if np.min(column) < 0:
            column = (column-mean)*1./ std
        elif np.max(column) > 1.0:                                                                                                                                        
            column = column *1./ mean
    
        X[:, j] = column
    
    return X

def return_best_chi2dof(tobs):
    """
    Returns the most fitting value for dof assuming tobs follows a chi2_dof distribution,
    computed with a Kolmogorov-Smirnov test, removing NANs and negative values.
    Parameters
    ----------
    tobs : np.ndarray
        observations
    Returns
    -------
        best : tuple
            tuple with best dof and corresponding chi2 test result
    """
    
    
    dof_range = np.arange(np.nanmedian(tobs) - 10, np.nanmedian(tobs) + 10, 0.1)
    
    ks_tests = []
    
    for dof in dof_range:
        
        test = kstest(tobs, lambda x:chi2.cdf(x, df=dof))[0]
        
        ks_tests.append((dof, test))
        
    ks_tests = [test for test in ks_tests if test[1] != 'nan'] # remove nans
    
    ks_tests = [test for test in ks_tests if test[0] >= 0] # retain only positive dof
        
    best = min(ks_tests, key = lambda t: t[1]) # select best dof according to KS test result
        
    return best

In [9]:
def emp_zscore(t0,t1):
    if max(t0) <= t1:
        p_obs = 1 / len(t0)
        Z_obs = round(norm.ppf(1 - p_obs),2)
        return Z_obs
    else:
        p_obs = np.count_nonzero(t0 >= t1) / len(t0)
        Z_obs = round(norm.ppf(1 - p_obs),2)
        return Z_obs

def chi2_zscore(t1, dof):
    p = chi2.cdf(float('inf'),dof)-chi2.cdf(t1,dof)
    return norm.ppf(1 - p)

In [10]:
def err_bar(hist, n_samples):
    
    bins_counts = hist[0]
    bins_limits = hist[1]
    
    x   = 0.5*(bins_limits[1:] + bins_limits[:-1])
    
    bins_width = 0.5*(bins_limits[1:] - bins_limits[:-1])
    err = np.sqrt(np.array(bins_counts)/(n_samples*np.array(bins_width)))
    
    return x, err

In [11]:
def plot_data(data, label, name=None, dof=None, out_path=None, title=None,
                 density=True, bins=10,
                 c='mediumseagreen', e='darkgreen'):
    """
    Plot reference vs new physics t distribution
    Parameters
    ----------
    data : np.ndarray or list
        (N_toy,) array of observed test statistics
    dof : int 
        degrees of freedom of the chi-squared distribution
    name : string
        filename for the saved figure
    out_path : string, optional
        output path where the figure will be saved. The default is ./fig.
    title : string
        title of the plot
    density : boolean
        True to normalize the histogram, false otherwise.
    bins : int or string, optional
        bins for the function plt.hist(). The default is 'fd'.
    Returns
    -------
    plot
    """
    
 
    plt.figure(figsize=(10,7))
    plt.style.use('classic')
    

    hist = plt.hist(data, bins = bins, color=c, edgecolor=e,
                        density=density, label = str(label))
    x_err, err = err_bar(hist, data.shape[0])
    plt.errorbar(x_err, hist[0], yerr = err, color=e, marker='o', ms=6, ls='', lw=1,
                 alpha=0.7)
    

    plt.ylim(bottom=0)
    
    # results data
    md_t = round(np.median(data), 2)
    if dof:
        z_chi2 = round(chi2_zscore(np.median(data),dof=dof),2)
    
    if dof:
        res = "md t = {} \nZ_chi2 = {}".format(md_t,z_chi2)
    else:
        res = "md t = {}".format(md_t)

    # plot chi2 and set xlim
    if dof:
        chi2_range = chi2.ppf(q=[0.00001,0.999], df=dof)
        x = np.arange(chi2_range[0], chi2_range[1], .05)
        chisq = chi2.pdf(x, df=dof)       
        plt.plot(x, chisq, color='#d7191c', lw=2, label='$\chi^2(${}$)$'.format(dof))
        xlim = (min(chi2_range[0], min(data)-5), max(chi2_range[1], max(data)+5))
        plt.xlim(chi2_range)
    else:
        xlim = (min(data)-5, max(data)+5)
        plt.xlim(xlim)


    if title:
        plt.title(title, fontsize=20)
    
    plt.ylabel('P(t)', fontsize=20)
    plt.xlabel('t', fontsize=20)
    
    # Axes ticks
    ax = plt.gca()
    
    plt.legend(loc ="upper right", frameon=True, fontsize=18)
    
    ax.text(0.75, 0.65, res, color='black', fontsize=12,
        bbox=dict(facecolor='none', edgecolor='black', boxstyle='round,pad=.5'),transform = ax.transAxes)
    
    plt.tight_layout()
    
    if out_path:
        plt.savefig(out_path+"/data_{}.pdf".format(name), bbox_inches='tight')
    
    plt.show()
        
    plt.close()

In [12]:
def plot_ref_data(ref, data, name=None, dof=None, out_path=None, title=None,
                 density=True, bins=10,
                 c_ref='#abd9e9', e_ref='#2c7bb6', c_sig='#fdae61', e_sig='#d7191c'):
    """
    Plot reference vs new physics t distribution
    Parameters
    ----------
    T_ref : np.ndarray or list
        (N_toy,) array of observed test statistics in the reference hypothesis
    T_sig : np.ndarray or list
        (N_toy,) array of observed test statistics in the New Physics hypothesis
    dof : int 
        degrees of freedom of the chi-squared distribution
    name : string
        filename for the saved figure
    out_path : string, optional
        output path where the figure will be saved. The default is ./fig.
    title : string
        title of the plot
    density : boolean
        True to normalize the histogram, false otherwise.
    bins : int or string, optional
        bins for the function plt.hist(). The default is 'fd'.
    Returns
    -------
    plot
    """
    
 
    plt.figure(figsize=(10,7))
    plt.style.use('classic')
    #set uniform bins across all data points
    bins = np.histogram(np.hstack((ref,data)), bins = bins)[1]
    
    # reference
    hist_ref = plt.hist(ref, bins = bins, color=c_ref, edgecolor=e_ref,
                        density=density, label = 'Reference')
    x_err, err = err_bar(hist_ref, ref.shape[0])
    plt.errorbar(x_err, hist_ref[0], yerr = err, color=e_ref, marker='o', ms=6, ls='', lw=1,
                 alpha=0.7)
    # data
    hist_sig = plt.hist(data, bins = bins, color=c_sig, edgecolor=e_sig,
                        alpha=0.7, density=density, label='Data')
    x_err, err = err_bar(hist_sig, data.shape[0])
    plt.errorbar(x_err, hist_sig[0], yerr = err, color=e_sig, marker='o', ms=6, ls='', lw=1,
                 alpha=0.7)
    

    plt.ylim(bottom=0)
    
    # results data
    md_tref = round(np.median(ref), 2)
    md_tdata = round(np.median(data), 2)
    max_zemp = emp_zscore(ref,np.max(ref))
    zemp = emp_zscore(ref,np.median(data))
    if dof:
        z_chi2 = round(chi2_zscore(np.median(data),dof=dof),2)
    
    if dof:
        res = "md t_ref = {} \nmd t_data = {} \nmax Z_emp = {}  \nZ_emp = {} \nZ_chi2 = {}".format(
            md_tref,
            md_tdata,
            max_zemp,
            zemp,
            z_chi2
        )
    else:
        res = "md tref = {} \nmd tdata = {} \nmax Zemp = {} \nZemp = {}".format(
            md_tref,
            md_tdata,
            max_zemp,
            zemp
        )

    # plot chi2 and set xlim
    if dof:
        chi2_range = chi2.ppf(q=[0.00001,0.999], df=dof)
        #r_len = chi2_range[1] - chi2_range[0]
        x = np.arange(chi2_range[0], chi2_range[1], .05)
        chisq = chi2.pdf(x, df=dof)       
        plt.plot(x, chisq, color='#d7191c', lw=2, label='$\chi^2(${}$)$'.format(dof))
        xlim = (min(chi2_range[0], min(ref)-1), max(chi2_range[1], max(data)+1))
        plt.xlim(xlim)
    else:
        xlim = (min(ref)-1, max(data)+1)
        plt.xlim(xlim)


    if title:
        plt.title(title, fontsize=20)
    
    plt.ylabel('P(t)', fontsize=20)
    plt.xlabel('t', fontsize=20)
    
    # Axes ticks
    ax = plt.gca()
    
    plt.legend(loc ="upper right", frameon=True, fontsize=18)
    
    ax.text(0.75, 0.55, res, color='black', fontsize=12,
        bbox=dict(facecolor='none', edgecolor='black', boxstyle='round,pad=.5'),transform = ax.transAxes)
    
    plt.tight_layout()
    
    if out_path:
        plt.savefig(out_path+"/refdata_{}.pdf".format(name), bbox_inches='tight')
        
    plt.close()

In [13]:
def get_template_roc_curve(ax, title,fs,random=True):
    
    ax.set_title(title, fontsize=fs)
    ax.set_xlim([-0.01, 1.01])
    ax.set_ylim([-0.01, 1.01])
    
    ax.set_xlabel('False Positive Rate', fontsize=fs)
    ax.set_ylabel('True Positive Rate', fontsize=fs)
    
    if random:
        ax.plot([0, 1], [0, 1],'r--',label="AUC ROC Random = 0.5")
        
def get_template_pr_curve(ax, title,fs, baseline=0.5):
    ax.set_title(title, fontsize=fs)
    ax.set_xlim([-0.01, 1.01])
    ax.set_ylim([-0.01, 1.01])
    
    ax.set_xlabel('Recall (True Positive Rate)', fontsize=fs)
    ax.set_ylabel('Precision', fontsize=fs)
    
    ax.plot([0, 1], [baseline, baseline],'r--',label='AP Random = 0.5')

In [14]:
def metrics_plots(data_path, output_path, N_0, N0, NS, flk_config, df=10, savefig=True):
  '''
  plotting the AUC ROC, and PR curves
  '''

  weight = N0/N_0
  # loading data
  df = pd.read_csv(data_path)

    
  # dividing the dataset
  fraud_df = df.loc[df['Class'] == 1]
  non_fraud_df = df.loc[df['Class'] == 0]

  dim = 2
  rng = np.random.default_rng(123)

  # Shuffling the data
  df = df.sample(frac=1, random_state=123)
  fraud_df = fraud_df.sample(frac=1,random_state=123)
  non_fraud_df = non_fraud_df.sample(frac=1,random_state=123)

  N0p = rng.poisson(lam=N0)
  NSp = rng.poisson(lam=NS) # if data contains anomalies
  N = N_0 + N0p + NSp

  X = np.zeros(shape=(N,dim))
  X[:N_0+N0p,:] = non_fraud_df[['V11','V14']].sample(n= N_0+N0p, replace=False, random_state=123) # ref and bkg
  X[N_0+N0p:,:] = fraud_df[['V11','V14']].sample(n= NSp, replace=False, random_state=123) # signal
  # initialize labes
  Y = np.zeros(shape=(N,1))
  Y[N_0:,:] = np.ones((N0p+NSp,1)) # flip data labels to one
  X = np.array(X)

  preds = trainer(X,Y,flk_config)

  curve, ax = plt.subplots(2, 1, figsize=(15,10))

  # AUC ROC curve

  FPR_list, TPR_list, threshold = metrics.roc_curve(Y, preds)
  ROC_AUC = metrics.auc(FPR_list, TPR_list)
    
  get_template_roc_curve(ax[0],title='Receiver Operating Characteristic Curve',fs=15)

  ax[0].plot(FPR_list, TPR_list, 'b', label = 'AUC ROC = {0:0.3f}'.format(ROC_AUC))
  ax[0].legend(loc = 'upper left',bbox_to_anchor=(1.05, 1))



  # PR (Precision Recall) curve

  get_template_pr_curve(ax[1], "Precision Recall (PR) Curve", fs=15, baseline = sum(np.array(Y))/len(np.array(Y)))

  precision, recall, threshold = metrics.precision_recall_curve(Y, preds)
  precision=precision[::-1]
  recall=recall[::-1]

  AP = metrics.average_precision_score(Y, preds)

  ax[1].step(recall, precision, 'b', label = 'AP = {0:0.3f}'.format(AP))
  ax[1].legend(loc = 'upper left',bbox_to_anchor=(1.05, 1))
  plt.subplots_adjust(wspace=0.5, hspace=0.8)

  if output_path:
      plt.savefig(output_path+"/metric_plots.pdf", bbox_inches='tight')
    
  plt.close()

In [15]:
def plot_reconstruction(df, data, weight_data, ref, weight_ref, t_obs, ref_preds,
                        save=False, save_path='', file_name=''):
    '''
    Reconstruction of the data distribution learnt by the model.
    
    df:              (int) chi2 degrees of freedom
    data:            (numpy array, shape (None, n_dimensions)) data training sample (label=1)
    weight_data:     (numpy array, shape (None,)) weights of the data sample (default ones)
    ref:             (numpy array, shape (None, n_dimensions)) reference training sample (label=0)
    weight_ref:      (numpy array, shape (None,)) weights of the reference sample
    tau_OBS:         (float) value of the tau term after training
    output_tau_ref:  (numpy array, shape (None, 1)) tau prediction of the reference training sample after training
    feature_labels:  (list of string) list of names of the training variables
    bins_code:       (dict) dictionary of bins edge for each training variable (bins_code.keys()=feature_labels)
    xlabel_code:     (dict) dictionary of xlabel for each training variable (xlabel.keys()=feature_labels)
    ymax_code:       (dict) dictionary of maximum value for the y axis in the ratio panel for each training variable (ymax_code.keys()=feature_labels)
    delta_OBS:       (float) value of the delta term after training (if not given, only tau reconstruction is plotted)
    output_delta_ref:(numpy array, shape (None, 1)) delta prediction of the reference training sample after training (if not given, only tau reconstruction is plotted)
    '''
    # used to regularize empty reference bins
    eps = 1e-10 

    weight_ref = np.ones(len(ref))*weight_ref
    weight_data = np.ones(len(data))*weight_data

    # Dimention
    dim = data.shape[-1]
    
    Zscore=norm.ppf(chi2.cdf(t_obs, df))

    for i in range(dim):

      bins = np.linspace(0,1.5,24)
      plt.rcParams["font.family"] = "serif"
      plt.style.use('classic')
      fig = plt.figure(figsize=(8, 8)) 
      fig.patch.set_facecolor('white')  
      ax1= fig.add_axes([0.1, 0.43, 0.8, 0.5])        
      hD = plt.hist(data[:, i],weights=weight_data, bins=bins, label='DATA', color='black', lw=1.5, histtype='step', zorder=2)
      hR = plt.hist(ref[:, i], weights=weight_ref, color='#a6cee3', ec='#1f78b4', bins=bins, lw=1, label='REFERENCE', zorder=1)
      hN = plt.hist(ref[:, i], weights=np.exp(ref_preds[:, 0])*weight_ref, histtype='step', bins=bins, lw=0)
    
      plt.errorbar(0.5*(bins[1:]+bins[:-1]), hD[0], yerr= np.sqrt(hD[0]), color='black', ls='', marker='o', ms=5, zorder=3)
      plt.scatter(0.5*(bins[1:]+bins[:-1]),  hN[0], edgecolor='black', label='RECO', color='#b2df8a', lw=1, s=30, zorder=4)

      font = font_manager.FontProperties(family='serif', size=16)
      l    = plt.legend(fontsize=18, prop=font, ncol=2)
      font = font_manager.FontProperties(family='serif', size=18) 
      title  = 't='+str(np.around(t_obs, 2))
      
      title += ', Z-score='+str(np.around(Zscore, 2))
      l.set_title(title=title, prop=font)
      plt.tick_params(axis='x', which='both',    labelbottom=False)
      plt.yticks(fontsize=16, fontname='serif')
      plt.xlim(0, 1.5)
      plt.ylabel("events", fontsize=22, fontname='serif')
      plt.yscale('log')
      ax2 = fig.add_axes([0.1, 0.1, 0.8, 0.3]) 
      x   = 0.5*(bins[1:]+bins[:-1])
      plt.errorbar(x, hD[0]/(hR[0]+eps), yerr=np.sqrt(hD[0])/(hR[0]+eps), ls='', marker='o', label ='DATA/REF', color='black')
      plt.plot(x, hN[0]/(hR[0]+eps), label ='RECO', color='#b2df8a', lw=3)

      font = font_manager.FontProperties(family='serif', size=16)
      plt.legend(fontsize=18, prop=font)
      plt.xlabel('x', fontsize=22, fontname='serif')
      plt.ylabel("ratio", fontsize=22, fontname='serif')

      plt.yticks(fontsize=16, fontname='serif')
      plt.xticks(fontsize=16, fontname='serif')
      plt.xlim(bins[0], bins[-1])
      plt.ylim(0,10)
      plt.grid()
      if save:
          os.makedirs(save_path, exist_ok=True)
          fig.savefig(save_path + 'dim'+ '-'+ '{}'.format(i+1)+ file_name)
      plt.show()
      plt.close()

    return

In [16]:
# for 2-D with plot-reconstruction

def run_toys(sig, data_path, output_path, N_0, N0, NS, flk_config, toys=np.arange(100), df=10, savefig=True):

    '''
    type of signal: "NP0", "NP1", "NP2", "NP3"
    output_path: directory (inside ./runs/) where to save results
    N_0: size of ref sample
    N0: expected num of bkg events
    NS: expected num of signal events
    flk_config: dictionary of logfalkon parameters
    toys: numpy array with seeds for toy generation
    plots_freq: how often to plot inputs with learned reconstructions
    df: degree of freedom of chi^2 for plots
    '''

    output_path = "./runs/" + output_path
    os.makedirs(output_path, exist_ok=True)

    #save config file (temporary solution)
    with open(output_path+"/flk_config.txt","w") as f:
        f.write( str(flk_config) )

    weight = N0/N_0
    # loading data
    samples = pd.read_csv(data_path)

    
    # dividing the dataset
    fraud_df = samples.loc[samples['Class'] == 1]
    non_fraud_df = samples.loc[samples['Class'] == 0]

    dim = 2

    for i in toys:

        st_time = time.time()

        rng = np.random.default_rng(i)

        # Shuffling the data
        samples = samples.sample(frac=1, random_state=i)
        fraud_df = fraud_df.sample(frac=1,random_state=i)
        non_fraud_df = non_fraud_df.sample(frac=1,random_state=i)

        N0p = rng.poisson(lam=N0)
        if sig!="NP0": NSp = rng.poisson(lam=NS) # if data contains anomalies
        else: NSp = 0

        N = N_0 + N0p + NSp

        print("[--] Toy {}: ".format(i))
        # build training set
        # initialize dataset
        # fill with ref, bkg and data
        if sig=="NP0":
            X = non_fraud_df[['V11','V14']].sample(n= N, replace=False, random_state=i) # both reference and data contain only bkg events (no NP component)
        else:
            X = np.zeros(shape=(N,dim))
            X[:N_0+N0p,:] = non_fraud_df[['V11','V14']].sample(n= N_0+N0p, replace=False, random_state=i) # ref and bkg
            X[N_0+N0p:,:] = fraud_df[['V11','V14']].sample(n= NSp, replace=False, random_state=i) # signal
        # initialize labes
        Y = np.zeros(shape=(N,1))
        Y[N_0:,:] = np.ones((N0p+NSp,1)) # flip data labels to one
        X = np.array(X)
        
        
        # in this 1D case, there is no need to standardize
        #Xoriginal = X.copy()
        #X = standardize(X)

        # learn_t
        flk_config['seed']=i # select different centers for different toys

        preds = trainer(X,Y,flk_config)

        t = compute_t(preds,Y,weight)

        dt = round(time.time()-st_time,2)

        with open(output_path+"t.txt", 'a') as f:
            f.write('{},{}\n'.format(i,t))

In [17]:
def get_results(ref, data, dof=None):
  md_tref = round(np.median(ref), 2)
  md_tdata = round(np.median(data), 2)
  max_zemp = emp_zscore(ref,np.max(ref))
  zemp = emp_zscore(ref,np.median(data))
  if dof:
      z_chi2 = round(chi2_zscore(np.median(data),dof=dof),2)

  return {"md_tref":md_tref,
          "md_tdata":md_tdata,
          "max_zemp":max_zemp,
          "zemp":zemp,
          "dof":dof,
          "z_chi2":z_chi2}

In [18]:
grid_parm = ["N_0","N0","M","NS"]

In [19]:
import itertools

flag = 0
parameters = {"N_0":[1000,2000,3000],"N0":[600,1000,2000],"M":[300,500,800],"NS":[2,5,10,20]}
combinations = itertools.product(*(parameters[Name] for Name in parameters.keys()))
for row in list(combinations):
  if row[2]<row[0] and row[2]<row[1]:
    flag += 1

print(flag)

96


In [20]:
import itertools

def GridSearch(parms):

  res = []
  combinations = itertools.product(*(parms[Name] for Name in parms.keys()))
  for row in list(combinations):

    # M should be less than N_0 and N0
    if row[2]<row[0] and row[2]<row[1]:
      # parameters
      flk_sigma = 3
      lam = 1e-7
      N_0, N0, M, NS = row[0], row[1], row[2], row[3]
      weight = N0/N_0

      flk_config = get_logflk_config(M,flk_sigma,[lam],weight=weight,iter=[100],seed=None,cpu=True)

      # Running the codes on reference dataset 
      NS_temp = 0
      run_toys("NP0", "drive/MyDrive/CreditCards/creditcard.csv", "./reference/", N_0, N0, NS_temp, flk_config, toys=np.arange(100), df= 22)

      # Best DoF
      best = return_best_chi2dof(np.loadtxt('/content/runs/reference/t.txt',delimiter=',')[:,1])

      # Running the codes on all data
      run_toys("NP1", "drive/MyDrive/CreditCards/creditcard.csv", "./NP1/", N_0, N0, NS, flk_config, toys=np.arange(100), df= best[0])

      # Collecting the important results for comparison
      results = get_results(np.loadtxt('/content/runs/reference/t.txt',delimiter=',')[:,1],np.loadtxt('/content/runs/NP1/t.txt',delimiter=',')[:,1],dof=best[0])

      # adding the parameters info to the collected data for comparison
      results.update({"N_0": N_0, "N0": N0, "M": M, "NS": NS})
      # Updating the whole Results
      res.append(results)

  return res

In [21]:
parameters = {"N_0":[1000,2000,3000],"N0":[600,1000],"M":[300,500,800],"NS":[2,5,10]}
my_dict = GridSearch(parameters)
res_df = pd.DataFrame(my_dict)

Streaming output truncated to the last 5000 lines.
[--] Toy 0: 
Iteration 0 - penalty 1.000000e-07 - sub-iterations 100
[--] Toy 1: 
Iteration 0 - penalty 1.000000e-07 - sub-iterations 100
[--] Toy 2: 
Iteration 0 - penalty 1.000000e-07 - sub-iterations 100
[--] Toy 3: 
Iteration 0 - penalty 1.000000e-07 - sub-iterations 100
[--] Toy 4: 
Iteration 0 - penalty 1.000000e-07 - sub-iterations 100
[--] Toy 5: 
Iteration 0 - penalty 1.000000e-07 - sub-iterations 100
[--] Toy 6: 
Iteration 0 - penalty 1.000000e-07 - sub-iterations 100
[--] Toy 7: 
Iteration 0 - penalty 1.000000e-07 - sub-iterations 100
[--] Toy 8: 
Iteration 0 - penalty 1.000000e-07 - sub-iterations 100
[--] Toy 9: 
Iteration 0 - penalty 1.000000e-07 - sub-iterations 100
[--] Toy 10: 
Iteration 0 - penalty 1.000000e-07 - sub-iterations 100
[--] Toy 11: 
Iteration 0 - penalty 1.000000e-07 - sub-iterations 100
[--] Toy 12: 
Iteration 0 - penalty 1.000000e-07 - sub-iterations 100
[--] Toy 13: 
Iteration 0 - penalty 1.000000e-07 

In [22]:
res_df

,md_tref,md_tdata,max_zemp,zemp,dof,z_chi2,N_0,N0,M,NS
0,44.48,50.27,2.33,0.61,44.079789,0.70,1000,600,300,2
1,44.48,51.52,2.58,0.67,44.079789,0.82,1000,600,300,5
2,44.48,56.58,2.71,1.13,44.079789,1.29,1000,600,300,10
3,44.49,53.93,2.81,0.91,44.094845,1.05,1000,600,500,2
4,44.49,55.06,2.88,0.95,44.094845,1.15,1000,600,500,5
5,44.49,56.81,2.94,1.15,44.094845,1.31,1000,600,500,10
6,45.20,57.82,2.98,0.90,45.701445,1.24,1000,1000,300,2
7,47.60,59.27,3.02,0.77,47.799621,1.16,1000,1000,300,5
8,48.97,61.37,3.06,0.81,49.269180,1.20,1000,1000,300,10
9,49.32,61.68,3.09,0.74,50.523318,1.10,1000,1000,500,2


In [23]:
res_df.to_csv("GridSearch_Results.csv",index=False)

In [26]:
res_df.sort_values(by='z_chi2')

,md_tref,md_tdata,max_zemp,zemp,dof,z_chi2,N_0,N0,M,NS
0,44.48,50.27,2.33,0.61,44.079789,0.70,1000,600,300,2
1,44.48,51.52,2.58,0.67,44.079789,0.82,1000,600,300,5
3,44.49,53.93,2.81,0.91,44.094845,1.05,1000,600,500,2
28,44.68,56.47,3.39,0.66,46.284151,1.06,2000,1000,800,5
27,44.86,56.74,3.38,0.66,46.462747,1.06,2000,1000,800,2
25,45.13,56.92,3.36,0.63,46.627279,1.06,2000,1000,500,5
24,45.16,57.16,3.35,0.61,46.763402,1.07,2000,1000,500,2
34,42.10,53.87,3.44,0.70,43.601482,1.09,3000,600,500,5
31,43.47,55.49,3.42,0.67,45.073235,1.09,3000,600,300,5
30,44.36,56.13,3.41,0.68,45.657803,1.09,3000,600,300,2


In [38]:
res_df.groupby(['N_0','N0','M']).mean()

md_tref   md_tdata  max_zemp      zemp        dof    z_chi2  \
N_0  N0   M                                                                    
1000 600  300  44.480000  52.790000  2.540000  0.803333  44.079789  0.936667   
          500  44.490000  55.266667  2.876667  1.003333  44.094845  1.170000   
     1000 300  47.256667  59.486667  3.020000  0.826667  47.590082  1.200000   
          500  49.826667  62.916667  3.116667  0.760000  51.395654  1.123333   
          800  51.976667  65.143333  3.190000  0.763333  53.312294  1.136667   
2000 600  300  50.033333  63.126667  3.243333  0.696667  51.433996  1.140000   
          500  47.780000  59.750000  3.290000  0.586667  48.377451  1.140000   
     1000 300  45.793333  57.940000  3.330000  0.593333  47.093860  1.110000   
          500  45.080000  57.080000  3.360000  0.630000  46.647515  1.073333   
          800  44.723333  56.666667  3.390000  0.673333  46.325959  1.070000   
3000 600  300  43.613333  55.620000  3.420000  0.680000  45.079408  1.103333   
          500  42.096667  53.986667  3.443333  0.700000  43.597544  1.106667   
     1000 300  41.420000  53.163333  3.466667  0.703333  42.786799  1.113333   
          500  41.036667  52.720000  3.486667  0.726667  42.302217  1.123333   
          800  40.350000  52.450000  3.506667  0.760000  41.852366  1.146667   

                     NS  
N_0  N0   M              
1000 600  300  5.666667  
          500  5.666667  
     1000 300  5.666667  
          500  5.666667  
          800  5.666667  
2000 600  300  5.666667  
          500  5.666667  
     1000 300  5.666667  
          500  5.666667  
          800  5.666667  
3000 600  300  5.666667  
          500  5.666667  
     1000 300  5.666667  
          500  5.666667  
          800  5.666667